# Hosting Capacity Spatial Mapping Models to Support Decision Making

## A Reduced Electrically-Equivalent Model of the IEEE European Low Voltage

### Panda Power

In [1]:
import pandas as pd
import pandapower as pp
import numpy as np
import os

data_dir = 'Modified_116_LV_CSV'

<div style="text-align: center;">
    <div style="background-color: white; display: inline-block; padding: 10px;">
        <img src="Modified_116_LV_CSV/Modified 116 LV feeder.png" width="600px">
    </div>
</div>

In [2]:
# Source data
source_df = pd.read_csv(os.path.join(data_dir,'Source.csv'), skiprows=1, sep='=')
source_dict = {i:float(row.iloc[0].split()[0]) for i, row in source_df.iterrows()}
print(source_dict)

source_dict['ISC1'] = source_dict['ISC1'] / 1000
source_dict['ISC3'] = source_dict['ISC3'] / 1000

# Transformer data
trafo_df = pd.read_csv(os.path.join(data_dir,'Transformer.csv'), skiprows=1, sep=';')
trafo_dict = trafo_df.iloc[0].to_dict()
print(trafo_dict)

{'Voltage': 11.0, 'pu': 1.05, 'ISC3': 3000.0, 'ISC1': 5.0}
{'Name': 'TR1', ' phases': 3, ' bus1': 'SourceBus', ' bus2': 1, ' kV_pri': 11, ' kV_sec': 0.416, ' MVA': 0.8, ' Conn_pri': ' Delta', ' Conn_sec': ' Wye', ' %XHL': 4, '% resistance': 0.4}


In [3]:
s_sc_mva = np.sqrt(3) * source_dict['Voltage'] * source_dict['ISC3'] #MVA

net = pp.create_empty_network()

bus_map = {}

hv_bus = pp.create_bus(net, name=trafo_dict[' bus1'], vn_kv=source_dict['Voltage'], type="b")
lv_bus = pp.create_bus(net, name=trafo_dict[' bus2'], vn_kv=trafo_dict[' kV_sec'], type="b")

bus_map[trafo_dict[' bus1']] = hv_bus
bus_map[trafo_dict[' bus2']] = lv_bus

pp.create_ext_grid(net, bus=hv_bus, vm_pu=source_dict['pu'], s_sc_max_mva=s_sc_mva, rx_max=0.1)

pp.create_transformer_from_parameters(
    net=net, hv_bus=hv_bus, lv_bus=lv_bus,
    sn_mva=trafo_dict[' MVA'],
    vn_hv_kv=trafo_dict[' kV_pri'],
    vn_lv_kv=trafo_dict[' kV_sec'],
    vk_percent=trafo_dict[' %XHL'],
    vkr_percent=trafo_dict['% resistance'],
    pfe_kw=0.0, i0_percent=0.0, shift_degree=0.0, 
    name=trafo_dict['Name']
)

0

In [4]:
lines_df = pd.read_excel(os.path.join(data_dir, "Lines.xlsx"), skiprows=1)
lines_df['Length'] = lines_df['Length'] / 1000 # m to km
lines_df['Units'] = 'km'
lines_df.head()

lineCodes_df = pd.read_csv(os.path.join(data_dir, "LineCodes.csv"), skiprows=1, sep=';')

all_bus_ids = np.unique(lines_df[['Bus1', 'Bus2']].values.ravel('K'))
for id in all_bus_ids:
    if id not in bus_map.keys():
        bus = pp.create_bus(net, name=id, vn_kv=trafo_dict[' kV_sec'], type="b")
        bus_map[id] = bus

to create a line from parameters `pp.create_line_from_parameters` we must first assign a `max_i_ka` per LineCode.

Here is the Standard Line Parameters for PandaPower:

In [6]:
StdLineCodes_df = pd.read_csv(os.path.join(data_dir, 'StandardLineCodes.csv'), sep=';')
# print(StdLineCodes_df.head())
StdLineCodes_df['max_i_ka'].describe()

count    51.000000
mean      0.432314
std       0.269581
min       0.105000
25%       0.250500
50%       0.350000
75%       0.498000
max       1.150000
Name: max_i_ka, dtype: float64

In [7]:
from sklearn.linear_model import LinearRegression

X = StdLineCodes_df[['r_ohm_per_km', 'x_ohm_per_km', 'c_nf_per_km']]
Y = StdLineCodes_df['max_i_ka']

x_test = lineCodes_df[['Name', 'R1', 'X1', 'C1']].set_index('Name')
x_test.columns = X.columns

model = LinearRegression()
model.fit(X, Y)

coefficients = dict(zip(X.columns, model.coef_))
intercept = model.intercept_

coefficients["intercept"] = intercept
print(coefficients)

y_pred = model.predict(x_test)
print(y_pred)

{'r_ohm_per_km': -0.027992220631350536, 'x_ohm_per_km': -0.862038384077252, 'c_nf_per_km': -0.0013628970743908564, 'intercept': 0.8237539896121313}
[0.71529451 0.71570356 0.72014921 0.74597276 0.75315532 0.76307509
 0.75006473 0.75094965]


In [8]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
x_test_scaled = scaler.fit_transform(x_test)

knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(X_scaled, Y)

y_pred = knn.predict(x_test_scaled)
print(y_pred)

lineCodes_df['max_i_ka'] = y_pred
lineCodes_df

[0.19    0.19    0.315   0.48425 0.48425 0.48425 0.48425 0.48425]


,Name,nphases,R1,X1,R0,X0,C1,C0,Units,max_i_ka
0,2c_.0225,3,1.257,0.0850,1.257,0.085,0,0,km,0.19000
1,2c_16,3,1.150,0.0880,1.200,0.088,0,0,km,0.19000
2,35_SAC_XSC,3,0.868,0.0920,0.760,0.092,0,0,km,0.31500
3,4c_.06,3,0.469,0.0750,1.581,0.091,0,0,km,0.48425
4,4c_.1,3,0.274,0.0730,0.959,0.079,0,0,km,0.48425
5,4c_.35,3,0.089,0.0675,0.319,0.076,0,0,km,0.48425
6,4c_70,3,0.446,0.0710,1.505,0.083,0,0,km,0.48425
7,4c_95_SAC_XC,3,0.322,0.0740,0.804,0.093,0,0,km,0.48425


In [ ]:
full_line_df = lines_df.merge(lineCodes_df, left_on="LineCode", right_on="Name", how="left")
full_line_df.head(10)

for _, line in full_line_df.head().iterrows():
    # print(line)
    pp.create_line_from_parameters(
        net, from_bus = line['Bus1'], to_bus = line['Bus2'],
        length_km = line['Length'],
        r_ohm_per_km=line["R1"],
        x_ohm_per_km=line["X1"],
        c_nf_per_km=line["C1"],
        max_i_ka=line["max_i_ka"],
        name=line["Name_x"]
    )

In [10]:
# Create Loads